<a href="https://colab.research.google.com/github/tg-bomze/ENTAR/blob/master/ENTAR_Eng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <b><font color="black" size="-5">.</font></b>
![alt text](https://raw.githubusercontent.com/tg-bomze/ENTAR/master/entar.png)

<b><font color="black" size="+3">Ensemble of Neural Tools for Animations Restoration</font></b>

<b><font color="red">P.S. Sorry for my bad English. I translate using Google Translator</font></b>

*↓ Open me ↓*

##**Put it all together:**
GitHub: [@tg-bomze](https://github.com/tg-bomze)

Telegram: [@bomze](https://t.me/bomze) 

Twitter: [@tg_bomze](https://twitter.com/tg_bomze)

*For all questions, Twitter.*

**Next are the blocks, divided into sections. The zero section is required. The rest is your choice.**

# <b><font color="gree" size="+3">0. Preparation for processing</b>
*↓ Click to open section ↓*

In [0]:
#@title ##**Install all the necessary libraries** { display-mode: "form" }
!pip install youtube_dl
from IPython.display import clear_output
from google.colab import files
import imageio
import youtube_dl
import cv2
import os
import torch
import fastai
from os import path
import numpy as np
import moviepy.editor as mpy
from moviepy.video.io.ffmpeg_writer import FFMPEG_VideoWriter
from pathlib import Path
import sys
import glob
from IPython import display as ipythondisplay
from IPython.display import Image as ipythonimage
torch.backends.cudnn.benchmark=True
%matplotlib inline

!rm -rf sample_data
clear_output()

In [0]:
#@title ##**Upload video** { display-mode: "form" }
#@markdown *Enter a link to the video below (for example, YouTube or Twitter), or leave the **source_url** field blank (in this case, you will be asked to download the video from your computer).*
source_url = '' #@param {type:"string"}

if source_url == '':
  uploaded = files.upload()
  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))
  file_name = "downloaded_video." + fn.split(".")[1]
  !mv -f $fn $file_name

else:
  try:
    ydl_opts = {
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4',
        'outtmpl': 'downloaded_video.mp4',
        }
    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
      ydl.download([source_url])
    file_name = 'downloaded_video.mp4'
  
  except BaseException:
    !wget $source_url
    fn = source_url.split('/')[-1]
    file_name = "downloaded_video." + fn.split(".")[1]
    !mv -f $fn $file_name

!cp -r downloaded_video.mp4 video.mp4
clear_output()
fps_of_video = int(cv2.VideoCapture(file_name).get(cv2.CAP_PROP_FPS))
frames_of_video = int(cv2.VideoCapture(file_name).get(cv2.CAP_PROP_FRAME_COUNT))
#@markdown *Downloading videos lasting longer than one minute is not recommended.*

#@markdown *If there is an error during execution, then run this block again*

**You can execute the following sections in any order (the main thing is to execute each block inside a section one by one).**

---
# <b><font color="gree" size="+3">1. Colorize video ([DeOldify](https://github.com/jantic/DeOldify))
*↓ Click to open section ↓*


Creator: [Jason Antic](https://twitter.com/citnaj)

In [0]:
#@title ##**Clone the repository and install the necessary dependencies** { display-mode: "form" }
!git clone https://github.com/jantic/DeOldify.git
%cd DeOldify
!pip install -r colab_requirements.txt
clear_output()

In [0]:
#@title ##**Upload the pre-trained model** { display-mode: "form" }
from deoldify.visualize import *
!mkdir 'models'
try:
  !rm -rf models/ColorizeVideo_gen.pth
  !gdown https://drive.google.com/uc?id=1-3ONnPYcX9fOqnY-pUKYGJOd6XeFon9X -O ./models/ColorizeVideo_gen.pth
except BaseException:
  !rm -rf models/ColorizeVideo_gen.pth
  !wget https://www.dropbox.com/s/336vn9y4qwyg9yz/ColorizeVideo_gen.pth?dl=0 -O ./models/ColorizeVideo_gen.pth
colorizer = get_video_colorizer()
clear_output()

In [0]:
#@title ##**Colorize the video** { display-mode: "form" }
#@markdown **21, 35** *- optimal values*
!rm -rf video
!mkdir 'video'
!mkdir 'video/source'
!cp -r ../video.mp4 video/source/video.mp4

render_factor = 21  #@param {type: "slider", min: 5, max: 44}
video_path = colorizer.colorize_from_file_name('video.mp4', render_factor)
clear_output()

If an error occurs while executing a block from above, then convert the video [HERE](https://convert-video-online.com/ru/) and start the entire script again.

In [0]:
#@title ##**Get result** { display-mode: "form" }
!cp -r video/result/video.mp4 ../colorized_video.mp4
what_next = 'play' #@param ["play", "download"]
if what_next == "play":
  display(mpy.ipython_display("../colorized_video.mp4", height=400, autoplay=1, loop=1, maxduration=600))
else:
  files.download('../colorized_video.mp4')

In [0]:
#@title ###**Optional block!** { display-mode: "form" }
#@markdown *If it seems to you that the quality of coloring is insufficient, then run this block, wait until it is complete and look at what value of **render_factor** you like the picture more. Next, run the previous coloring block again, setting the desired value.*
for i in range(10,45,2):
    colorizer.vis.plot_transformed_image('video/bwframes/video/00001.jpg', render_factor=i, display_render_factor=True, figsize=(8,8))

In [0]:
#@title ##**Exit DeOldify** { display-mode: "form" }
#@markdown *If you are going to continue to carry out the following points, then first of all, be sure to complete this block!*
!rm -rf ../video.mp4
!cp -r video/result/video.mp4 ../video.mp4
%cd ..
clear_output()


---
# <b><font color="gree" size="+3">2. Delete noise ([DeepRemaster](https://github.com/satoshiiizuka/siggraphasia2019_remastering))
*↓ Click to open section ↓*

Creator: [Satoshi Iizuka](http://iizuka.cs.tsukuba.ac.jp/index_eng.html)

In [0]:
#@title ##**Clone the repository** { display-mode: "form" }
!git clone https://github.com/satoshiiizuka/siggraphasia2019_remastering.git DeepRemaster
!cp -r video.mp4 DeepRemaster/
%cd DeepRemaster
clear_output()

In [0]:
#@title ##**Upload the pre-trained model** { display-mode: "form" }
try:
  !rm -rf model/remasternet.pth.tar
  !gdown https://drive.google.com/uc?id=1pB8C7c2EDUXNL0Ytk0NZ8q8bAj_nFykO
  !gdown https://drive.google.com/uc?id=1fJJK6i6jVGJmWhoHssuHMZFmfnULxe8S -O model/remasternet.pth.tar
  !bash ./checking_model.sh
except BaseException:
  !rm -rf model/remasternet.pth.tar
  !bash ./download_model.sh
clear_output()

In [0]:
#@title ##**Remove frame noise** { display-mode: "form" }
!python remaster.py --input video.mp4 --disable_colorization --gpu
clear_output()

In [0]:
#@title ##**Get result** { display-mode: "form" }
!rm -rf ../video.mp4
!cp -r video_out.mp4 ../video.mp4
!cp -r video_out.mp4 ../denoise_video.mp4
what_next = 'play' #@param ["play", "download"]
if what_next == "play":
  display(mpy.ipython_display("../denoise_video.mp4", height=400, autoplay=1, loop=1, maxduration=600))
else:
  files.download('../denoise_video.mp4')

In [0]:
#@title ##**Exit DeepRemaster** { display-mode: "form" }
#@markdown *If you are going to continue to carry out the following points, then first of all, be sure to complete this block!*
%cd ..
clear_output()


---
# <b><font color="gree" size="+3">3. Frames interpolation ([VFIASC](https://github.com/sniklaus/sepconv-slomo))

*(You can find another interpolation method below in the "Advanced sections" section. It runs faster but less qualitatively.)*

*↓ Click to open section ↓*

Creator: [Simon Niklaus](http://sniklaus.com/about/welcome)

In [0]:
#@title ##**Clone the repository and download the necessary components** { display-mode: "form" }
!git clone https://github.com/sniklaus/sepconv-slomo.git VFIASC
%cd VFIASC
!rm -rf network-l1.pytorch
!rm -rf network-lf.pytorch
try:
  !gdown https://drive.google.com/uc?id=1v77wNU8sYh0hBmljeGt5LzY9ehQU0elv
  !gdown https://drive.google.com/uc?id=1na11Ey0TB1KEDps4uEwyTBsDcmpo2kr6
except BaseException:
  !bash download.bash
clear_output()

In [0]:
#@title ##**Install the necessary dependencies** { display-mode: "form" }
#@markdown **This block is required. Execution time: ~ 12 minutes**
print('You can go for a coffee while.\n')
!pip install cupy
!pip install moviepy
clear_output()
print("It's time to do the next block.")

In [0]:
#@title ##**Interpolate frames** { display-mode: "form" }
#@markdown **How many times increase FPS:**
fps_boosting = 'x4' #@param ["x4", "x8"]

if fps_boosting == 'x4':
  !python run.py --model lf --video ../video.mp4 --out ./video_x4.mp4
  video_name = "video_x4.mp4"
elif fps_boosting == 'x8':
  !python run.py --model lf --video ../video.mp4 --out ./video_x4.mp4
  !python run.py --model lf --video ./video_x4.mp4 --out ./video_x8.mp4
  video_name = "video_x8.mp4"
clear_output()

mfps_of_video = int(cv2.VideoCapture(video_name).get(cv2.CAP_PROP_FPS))
mframes_of_video = int(cv2.VideoCapture(video_name).get(cv2.CAP_PROP_FRAME_COUNT))
#@markdown *I recommend installing* **x4**. *Otherwise, the execution of this block may be delayed or lead to an error. On top of that, the effect of SlowMo will be unnatural*

In [0]:
#@title ##**Get result** { display-mode: "form" }
!rm -rf ../video.mp4
!cp -r $video_name ../video.mp4
!cp -r $video_name ../fps_boosted_video.mp4
what_next = 'play' #@param ["play", "download"]
if what_next == "play":
  display(mpy.ipython_display("../fps_boosted_video.mp4", height=400, autoplay=1, loop=1, maxduration=600))
else:
  files.download("../fps_boosted_video.mp4")

In [0]:
#@title ##**Convert SlowMo video to high-frequency and automatically suggest downloading it to a computer** { display-mode: "form" }
!rm -rf output.mp4
!rm -rf slowed_movie_frames
!rm -rf ../video.mp4
!rm -rf ../fps_boosted_video.mp4

fr = int(fps_of_video*int(fps_boosting.split("x")[1]))
!mkdir 'slowed_movie_frames'
vidcap = cv2.VideoCapture(video_name)
success,image = vidcap.read()
count = 0
success = True
while success:
  cv2.imwrite("slowed_movie_frames/frame%09d.jpg" % count, image)
  success,image = vidcap.read()
  count += 1

staffs = []
img = os.listdir("slowed_movie_frames/")
img.sort()
clear_output()
for i in img:
  staffs.append("slowed_movie_frames/"+i)

staff = cv2.imread(staffs[0])
writer = cv2.VideoWriter(
    'output.mp4',
    cv2.VideoWriter_fourcc(*'MP4V'),   # codec (*'DIVX', *'MP4V', *'FMP4', *'MJPG', *'XVID', *'MP4S')
    fr,
    (staff.shape[1], staff.shape[0]),
    isColor=len(staff.shape) > 2)
for staff in map(cv2.imread, staffs):
    writer.write(staff)
writer.release()

clear_output()

pfps_of_video = int(cv2.VideoCapture("output.mp4").get(cv2.CAP_PROP_FPS))
pframes_of_video = int(cv2.VideoCapture("output.mp4").get(cv2.CAP_PROP_FRAME_COUNT))
!cp -r output.mp4 ../fps_boosted_video.mp4
files.download("../fps_boosted_video.mp4")

In [0]:
#@title ##**Exit VFIASC** { display-mode: "form" }
#@markdown *If you are going to continue to carry out the following points, then first of all, be sure to complete this block!*
!cp -r ../fps_boosted_video.mp4 ../video.mp4
%cd ..
clear_output()


---
# <b><font color="gree" size="+3">4. Upscale Resolution ([ESRGAN](https://github.com/xinntao/ESRGAN))

*↓ Click to open section ↓*

Creator: [Xintao Wang](https://xinntao.github.io)

In [0]:
#@title ##**Clone the repository and download the necessary components** { display-mode: "form" }
!git clone https://github.com/xinntao/ESRGAN.git
!cp -r video.mp4 ESRGAN/
%cd ESRGAN
!git checkout tags/old-arch
model_url = "https://www.dropbox.com/s/vouc15j8jjp2o5n/RRDB_ESRGAN_x4_old_arch.pth?dl=0"
!wget $model_url --content-disposition -P models
import architecture as arch
import os.path
!mkdir frames
!rm -rf results/baboon_ESRGAN.png
clear_output()

In [0]:
#@title ##**Split the video into frames** { display-mode: "form" }
frames_of_video = int(cv2.VideoCapture("video.mp4").get(cv2.CAP_PROP_FRAME_COUNT))
fps_of_video = int(cv2.VideoCapture("video.mp4").get(cv2.CAP_PROP_FPS))
vidcap = cv2.VideoCapture('video.mp4')
success,image = vidcap.read()
count = 0
success = True
while success:
  cv2.imwrite("frames/frame%09d.jpg" % count, image)
  success,image = vidcap.read()
  count += 1

In [0]:
#@title ##**Upscale resolution and improve frame quality** { display-mode: "form" }
#@markdown **How many times to upscale the resolution:**
upscale = 4 #@param {type: "slider", min: 4, max: 8}

device = torch.device('cuda')
model = arch.RRDB_Net(3, 3, 64, 23, gc=32, upscale=upscale, norm_type=None, act_type='leakyrelu', \
                        mode='CNA', res_scale=1, upsample_mode='upconv')
model.load_state_dict(torch.load('models/{:s}'.format('RRDB_ESRGAN_x4_old_arch.pth')), strict=True)
model.eval()
for k, v in model.named_parameters():
    v.requires_grad = False
model = model.to(device)

count_frames = 0

for path in glob.glob('frames/*'):
    base = os.path.splitext(os.path.basename(path))[0]
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    img = img * 1.0 / 255
    img = torch.from_numpy(np.transpose(img[:, :, [2, 1, 0]], (2, 0, 1))).float()
    img_LR = img.unsqueeze(0)
    img_LR = img_LR.to(device)

    output = model(img_LR).data.squeeze().float().cpu().clamp_(0, 1).numpy()
    output = np.transpose(output[[2, 1, 0], :, :], (1, 2, 0))
    output = (output * 255.0).round()
    path = 'results/{:s}_rlt.png'.format(base)
    cv2.imwrite(path, output)
    count_frames += 1
    clear_output()
    print("Processed: {} из {}".format(str(count_frames), str(frames_of_video)))
clear_output()

In [0]:
#@title ##**Collecting a video file** { display-mode: "form" }
frames = []
img = os.listdir("results/")
img.sort()
for i in img:
  frames.append(imageio.imread("results/"+i))
frames = np.array(frames)
imageio.mimsave("upscale_video.mp4", frames, fps=fps_of_video)

print('Сборка завершена')
!cp -r upscale_video.mp4 ../upscale_video.mp4
!cp -r upscale_video.mp4 ../video.mp4
clear_output()

In [0]:
#@title ##**Get result** { display-mode: "form" }
what_next = 'play' #@param ["play", "download"]
if what_next == "play":
  display(mpy.ipython_display("../upscale_video.mp4", height=400, autoplay=1, loop=1, maxduration=600))
else:
  files.download("../upscale_video.mp4")

In [0]:
#@title ##**Exit ESRGAN** { display-mode: "form" }
#@markdown *If you are going to continue to carry out the following points, then first of all, be sure to complete this block!*
%cd ..
clear_output()


# <b><font color="black" size="+3">Advanced sections
<font color="grey">These sections contain neural networks that are not included in the main list of video processing tools for various reasons.

*↓ Click to open additional sections. ↓*

---
## <b><font color="grey" size="+2">Increased frame resolution ([Fast-SRGAN](https://github.com/HasnainRaz/Fast-SRGAN))
*If you decide to execute this block, it is recommended that you do this to the ambassador of section* 2. Delete noise

*↓ Click to open section ↓*

Creator: [Hasnain Raza](https://github.com/HasnainRaz)

In [0]:
#@title ###**Clone the repository and install the necessary dependencies** { display-mode: "form" }
!git clone https://github.com/HasnainRaz/Fast-SRGAN.git
!cp -r video.mp4 Fast-SRGAN/
%cd Fast-SRGAN
!pip install -r requirements.txt
clear_output()

In [0]:
#@title ###**Generate high-resolution frames** { display-mode: "form" }
#@markdown *This block works very slowly, and the quality of the resulting images is very controversial, which is why I put this section into Advanced.*
!rm -rf hr_frames
!rm -rf frames
!mkdir 'hr_frames'
!mkdir 'frames'

vidcap = cv2.VideoCapture('video.mp4')
success,image = vidcap.read()
count = 0
success = True
while success:
  cv2.imwrite("frames/frame%09d.jpg" % count, image)
  success,image = vidcap.read()
  count += 1

!python infer.py --image_dir 'frames/' --output_dir 'hr_frames/'
clear_output()

In [0]:
#@title ###**Putting frames in a single video** { display-mode: "form" }

!rm -rf HR_video.mp4
!rm -rf ../HR_video.mp4
frames = []
img = os.listdir("hr_frames/")
img.sort()
for i in img:
  frames.append(imageio.imread("hr_frames/"+i))
frames = np.array(frames)
imageio.mimsave("HR_video.mp4", frames)

print('Assembly completed')
!cp -r HR_video.mp4 ../HR_video.mp4
clear_output()

In [0]:
#@title ###**Get result** { display-mode: "form" }
what_next = 'play' #@param ["play", "download"]
if what_next == "play":
  display(mpy.ipython_display("HR_video.mp4", height=400, autoplay=1, loop=1, maxduration=600))
else:
  files.download("HR_video.mp4")

In [0]:
#@title ###**Convert SlowMo video to high-frequency and automatically suggest downloading it to a computer** { display-mode: "form" }
fps_of_video = int(cv2.VideoCapture("video.mp4").get(cv2.CAP_PROP_FPS))

!rm -rf HR_video.mp4
!rm -rf ../HR_video.mp4
frames = []
img = os.listdir("hr_frames/")
img.sort()
clear_output()
for i in img:
  frames.append("hr_frames/"+i)
frames = np.array(frames)

frame = cv2.imread(frames[0])
writer = cv2.VideoWriter(
    'HR_video.mp4',
    cv2.VideoWriter_fourcc(*'MP4V'),   # codec (*'DIVX', *'MP4V', *'FMP4', *'MJPG', *'XVID', *'MP4S')
    fps_of_video, 
    (frame.shape[1], frame.shape[0]), 
    isColor=len(frame.shape) > 2)
for frame in map(cv2.imread, frames):
    writer.write(frame)
writer.release()

clear_output()
print('Conversion completed. Wait for the download to start.')
!cp -r HR_video.mp4 ../HR_video.mp4
files.download("HR_video.mp4")
clear_output()

In [0]:
#@title ###**Exit Fast-SRGAN** { display-mode: "form" }
#@markdown *If you are going to continue to carry out the following points, then first of all, be sure to complete this block!*
!rm -rf ../video.mp4
!cp -r HR_video.mp4 ../video.mp4
%cd ..
clear_output()


---
## <b><font color="grey" size="+2">Faster but lower quality frame interpolation ([VFI-CFT](https://github.com/MortenHannemose/pytorch-vfi-cft))
*↓ Click to open section ↓*

Creator: [Morten Hannemose](https://scholar.google.ru/citations?user=AH58CjUAAAAJ&hl=ru)

In [0]:
#@title ###**Clone a repository and download a pre-trained model** { display-mode: "form" }
!git clone https://github.com/MortenHannemose/pytorch-vfi-cft.git VFI-CFT
video = "video_2x_slow.mp4"
try:
  !rm -rf VFI_CFT_weights.pt.gz
  !rm -rf slow_movie.py
  !gdown https://drive.google.com/uc?id=1-5zS1XOhwVXJhD4JDZWmG6plC_IVVg8b
  !gdown https://drive.google.com/uc?id=1YinJeeUbTbDynD6kw6f6h76CAkRqFdHK
except BaseException:
  !rm -rf VFI_CFT_weights.pt.gz
  !gdown https://drive.google.com/uc?id=1rQAi0UXcaMcEo8_l4oSd8vJUVzp2dIIF
clear_output()

In [0]:
#@title ###**Interpolate frames** { display-mode: "form" }
#@markdown **How many times increase FPS:**
!rm -rf slowed_movie_frames
!rm -rf $video
!rm -rf video.mp4
!cp -r ../video.mp4 ./
fps_boosting = 2 #@param {type:"slider", min:2, max:8, step:1}

!python slow_movie.py -m video.mp4 -f $fps_boosting

video = "video_"+str(fps_boosting)+"x_slow.mp4"
frames = []
img = os.listdir("slowed_movie_frames/")
img.sort()
clear_output()
for i in img:
  frames.append(imageio.imread("slowed_movie_frames/"+i))
frames = np.array(frames)
imageio.mimsave("fps_boosted_video.mp4", frames)
#@markdown *More than x4 is not recommended. SlowMo effect will be unnatural*
clear_output()

nfps_of_video = int(cv2.VideoCapture("fps_boosted_video.mp4").get(cv2.CAP_PROP_FPS))
nframes_of_video = int(cv2.VideoCapture("fps_boosted_video.mp4").get(cv2.CAP_PROP_FRAME_COUNT))
nseconds_of_video = int(nframes_of_video/nfps_of_video)
print("Received video last: {} sec".format(str(nseconds_of_video)))
print("Total number of frames: {}".format(str(nframes_of_video)))
print("Frame rate: {} fps".format(str(nfps_of_video)))

In [0]:
#@title ###**Get result** { display-mode: "form" }
!rm -rf ../video.mp4
!cp -r fps_boosted_video.mp4 ../video.mp4
!cp -r fps_boosted_video.mp4 ../
what_next = 'play' #@param ["play", "download"]
if what_next == "play":
  display(mpy.ipython_display("../fps_boosted_video.mp4", height=400, autoplay=1, loop=1, maxduration=600))
else:
  files.download("../fps_boosted_video.mp4")

In [0]:
#@title ###**Exit VFI-CFT** { display-mode: "form" }
#@markdown *If you are going to continue to carry out the following points, then first of all, be sure to complete this block!*
%cd ..
clear_output()


---
## <b><font color="grey" size="+2">Bonus. Colorize photo and parallax effect ([DeOldify](https://github.com/jantic/DeOldify) + [3D Ken Burns](https://github.com/sniklaus/3d-ken-burns))</b>
This section can be performed very first. At the end, the resulting video can be restored in other sections. It is important that before going through other sections, you need to go to **"0. Preparation for processing"** and execute the block **"Install all the necessary libraries"**. Further, all other sections will function normally.

*↓ **Click to open section** ↓*

In [0]:
#@title ###**Upload photo** { display-mode: "form" }
#@markdown *Enter a link to the photo below, or leave the **source_url** field blank (in this case, you will be prompted to download the video from your computer).*
!rm -rf sample_data
source_url = 'http://www.worldfocus.ru/_ph/1/444433832.jpg' #@param {type:"string"}

if source_url == '':
  uploaded = files.upload()
  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))
  pic_name = "photo." + fn.split(".")[1]
  !mv -f $fn $pic_name

else:
  try:
    pic_name = "photo." + source_url.split('.')[-1]
    !wget $source_url -O $pic_name
  except BaseException:
    print('Something wrong. Try uploading photos from your computer.')

!cp -r photo.jpg downloaded_photo.jpg
clear_output()
#@markdown *If an error occurs during execution, then run this block again*

### <b><font color="aquamarine" size="+1">1) DeOldify - Colorize photo</b>
Creator: [Jason Antic](https://twitter.com/citnaj)

*↓ Click to open subsection ↓*

In [0]:
#@title ###**Clone the repository and install the necessary dependencies** { display-mode: "form" }
import torch
import fastai
!git clone https://github.com/jantic/DeOldify.git DeOldifyPhoto
%cd DeOldifyPhoto
!pip install -r colab_requirements.txt
clear_output()

In [0]:
#@title ###**Upload the pre-trained model** { display-mode: "form" }
from deoldify.visualize import *
!mkdir 'models'
try:
  !rm -rf models/ColorizeVideo_gen.pth
  !gdown https://drive.google.com/uc?id=1hJ_Ut8JQ6mxk0W1ChRmBXcju0oiMo1ZU -O ./models/ColorizeArtistic_gen.pth
except BaseException:
  !rm -rf models/ColorizeVideo_gen.pth
  !wget https://www.dropbox.com/s/zkehq1uwahhbc2o/ColorizeArtistic_gen.pth?dl=0 -O ./models/ColorizeArtistic_gen.pth
colorizer = get_image_colorizer(artistic=True)
clear_output()

In [0]:
#@title ###**Colorize photo** { display-mode: "form" }
#@markdown **21** *- optimal value*
!rm -rf ../colorized_photo.jpg
from google.colab import files
render_factor = 21  #@param {type: "slider", min: 5, max: 44}
image_path = colorizer.plot_transformed_image(path='../photo.jpg', render_factor=render_factor, compare=True)
!cp -r result_images/photo.jpg ../colorized_photo.jpg

#@markdown **Download:**
download_image = True #@param {type:"boolean"}
if download_image == True:
  files.download("result_images/photo.jpg")

In [0]:
#@title ###**Exit DeOldify** { display-mode: "form" }
#@markdown *If you are going to continue to carry out the following points, then first of all, be sure to complete this block!*
!cp -r result_images/photo.jpg ../photo.jpg
%cd ..
clear_output()


### <b><font color="aquamarine" size="+1">2) 3D Ken Burns - Parallax effect</b>

Creator: [Simon Niklaus](http://sniklaus.com/about/welcome)

*↓ Click to open subsection ↓*

In [0]:
#@title ###**Сlone the repository and download the necessary components** { display-mode: "form" }
import os
import imageio
import numpy as np
import moviepy.editor as mpy
from moviepy.video.io.ffmpeg_writer import FFMPEG_VideoWriter
%matplotlib inline
os.environ['CUDA_HOME'] = "/usr/local/cuda"
!pip install moviepy
!git clone https://github.com/sniklaus/3d-ken-burns.git 3D-Ken-Burns
%cd 3D-Ken-Burns
!mkdir 'results'
clear_output()

In [0]:
#@title ###**Upload the pre-trained models** { display-mode: "form" }
!rm -rf models/disparity-estimation.pytorch
!rm -rf models/disparity-refinement.pytorch
!rm -rf models/pointcloud-inpainting.pytorch
try:
  !gdown https://drive.google.com/uc?id=1-0M3GAZkiOexs7gqgWuORzqYAp00DeE- -O ./models/disparity-estimation.pytorch
  !gdown https://drive.google.com/uc?id=1-11KRkOKxqLmKOQk69PDMYbaem-3mH04 -O ./models/disparity-refinement.pytorch
  !gdown https://drive.google.com/uc?id=1-1-VjUQHVDwxtLY9xn65c3LpKCeIW6jH -O ./models/pointcloud-inpainting.pytorch
except BaseException:
  !wget -O ./3d-ken-burns/models/disparity-estimation.pytorch http://content.sniklaus.com/kenburns/network-disparity.pytorch
  !wget -O ./3d-ken-burns/models/disparity-refinement.pytorch http://content.sniklaus.com/kenburns/network-refinement.pytorch
  !wget -O ./3d-ken-burns/models/pointcloud-inpainting.pytorch http://content.sniklaus.com/kenburns/network-inpainting.pytorch
clear_output()

In [0]:
#@title ###**Create Parallax Effect** { display-mode: "form" }
!cp -r  ../photo.jpg ./images/photo.jpg
!rm images/README.md
!rm images/doublestrike.jpg
!for image in ./images/*; do python autozoom.py --in $image --out ./results/$(basename $image | cut -f1 -d '.').mp4; done
clear_output()
#@markdown *To save a video, right-click on it and select* **Save**
display(mpy.ipython_display("results/photo.mp4", height=400, autoplay=1, loop=1))

In [0]:
#@title ###**Exit 3D Ken Burns** { display-mode: "form" }
#@markdown *If you are going to continue to carry out the following points, then first of all, be sure to complete this block!*
!cp -r results/photo.mp4 ../parallax_video.mp4
!cp -r results/photo.mp4 ../video.mp4
%cd ..
clear_output()